In [1400]:
using LowLevelFEM, LinearAlgebra, SparseArrays

In [1401]:
function poissonMatrixVector(
    problem::Problem;
    coefficient::TensorField)

    @assert problem.pdim == problem.dim
    gmsh.model.setCurrent(problem.name)

    dim = problem.dim
    pdim = problem.pdim
    non = problem.non
    dof = pdim * non

    # elementwise tensor field (node values gathered per element)
    S = nodesToElements(coefficient)
    Se = Dict(zip(S.numElem, S.A))

    lengthOfIJV = LowLevelFEM.estimateLengthOfIJV(problem)
    I = Vector{Int}(undef, lengthOfIJV)
    J = Vector{Int}(undef, lengthOfIJV)
    V = Vector{Float64}(undef, lengthOfIJV)
    pos = 1

    for mat in problem.material
        for (edim, etag) in gmsh.model.getEntitiesForPhysicalName(mat.phName)

            elemTypes, elemTags, elemNodeTags =
                gmsh.model.mesh.getElements(edim, etag)

            for it in eachindex(elemTypes)
                et = elemTypes[it]
                _, _, order, numNodes, _, _ =
                    gmsh.model.mesh.getElementProperties(et)

                intPoints, intWeights =
                    gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))

                # --- get Lagrange shape functions for proper interpolation
                _, hfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, intPoints, "Lagrange")
                H = reshape(hfun, numNodes, :)

                # --- gradients
                _, dfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, intPoints, "GradLagrange")
                ∇h = reshape(dfun, :, length(intWeights))

                for (e, elem) in enumerate(elemTags[it])
                    nodeTags = elemNodeTags[it][(e-1)*numNodes+1:e*numNodes]

                    jac, jacDet, _ =
                        gmsh.model.mesh.getJacobian(elem, intPoints)
                    Jac = reshape(jac, 3, :)

                    Ke = zeros(pdim * numNodes, pdim * numNodes)

                    # tensor at element nodes (9×numNodes)
                    Snode = Se[elem]

                    for k in eachindex(intWeights)
                        invJ = inv(Jac[1:dim, 3k-2:3k])'
                        w = jacDet[k] * intWeights[k]

                        # --- interpolate S to Gauss point using shape functions
                        Sgp = zeros(dim, dim)
                        for a in 1:numNodes
                            Sa = reshape(Snode[9a-8:9a, 1], dim, dim)
                            Sgp .+= H[a, k] * Sa
                        end

                        for a in 1:numNodes, b in 1:numNodes
                            ∇Na = invJ * ∇h[3a-2:3a-(3-dim), k]
                            ∇Nb = invJ * ∇h[3b-2:3b-(3-dim), k]

                            for i in 1:dim, j in 1:dim
                                ia = (a - 1) * pdim + i
                                ib = (b - 1) * pdim + j
                                Ke[ia, ib] += ∇Na[i] * Sgp[i, j] * ∇Nb[j] * w
                            end
                        end
                    end

                    # scatter
                    for a in 1:pdim*numNodes, b in 1:pdim*numNodes
                        I[pos] = (nodeTags[div(a - 1, pdim)+1] - 1) * pdim + mod(a - 1, pdim) + 1
                        J[pos] = (nodeTags[div(b - 1, pdim)+1] - 1) * pdim + mod(b - 1, pdim) + 1
                        V[pos] = Ke[a, b]
                        pos += 1
                    end
                end
            end
        end
    end

    resize!(I, pos - 1)
    resize!(J, pos - 1)
    resize!(V, pos - 1)

    K = sparse(I, J, V, dof, dof)
    dropzeros!(K)
    return SystemMatrix(K, problem)
end


poissonMatrixVector (generic function with 1 method)

In [1402]:
function gradDivMatrixF(
    problem::Problem;
    coefficient::Union{Number,ScalarField},
    F::TensorField)

    @assert problem.pdim == problem.dim
    gmsh.model.setCurrent(problem.name)

    dim = problem.dim
    pdim = problem.pdim
    non = problem.non
    dof = pdim * non

    # --- elementwise nodal tensors for F
    Fe = nodesToElements(F)
    Fe_map = Dict(zip(Fe.numElem, Fe.A))

    # --- elementwise nodal scalars for λ (if needed)
    λ_is_scalarfield = !(coefficient isa Number)
    if λ_is_scalarfield
        λe = nodesToElements(coefficient)
        λmap = Dict(zip(λe.numElem, λe.A))   # each entry: (numNodes×1) vector per element
    else
        λconst = Float64(coefficient)
    end

    lengthOfIJV = LowLevelFEM.estimateLengthOfIJV(problem)
    I = Vector{Int}(undef, lengthOfIJV)
    J = Vector{Int}(undef, lengthOfIJV)
    V = Vector{Float64}(undef, lengthOfIJV)
    pos = 1

    for mat in problem.material
        for (edim, etag) in gmsh.model.getEntitiesForPhysicalName(mat.phName)

            elemTypes, elemTags, elemNodeTags =
                gmsh.model.mesh.getElements(edim, etag)

            for it in eachindex(elemTypes)
                et = elemTypes[it]
                _, _, order, numNodes, _, _ =
                    gmsh.model.mesh.getElementProperties(et)

                intPoints, intWeights =
                    gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))

                # --- Lagrange shape functions (for proper interpolation)
                _, hfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, intPoints, "Lagrange")
                H = reshape(hfun, numNodes, :)

                # --- GradLagrange
                _, dfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, intPoints, "GradLagrange")
                ∇h = reshape(dfun, :, length(intWeights))

                for (e, elem) in enumerate(elemTags[it])
                    nodeTags = elemNodeTags[it][(e-1)*numNodes+1:e*numNodes]

                    jac, jacDet, _ =
                        gmsh.model.mesh.getJacobian(elem, intPoints)
                    Jac = reshape(jac, 3, :)

                    Ke = zeros(pdim * numNodes, pdim * numNodes)

                    # nodal data for this element
                    Felem = Fe_map[elem]
                    if λ_is_scalarfield
                        λa = λmap[elem][:, 1]  # length = numNodes
                    end

                    for k in eachindex(intWeights)
                        invJ = inv(Jac[1:dim, 3k-2:3k])'
                        w = jacDet[k] * intWeights[k]

                        # --- interpolate F to GP using shape functions
                        Fgp = zeros(dim, dim)
                        for a in 1:numNodes
                            Na = H[a, k]
                            Fgp .+= Na * reshape(Felem[9a-8:9a, 1], dim, dim)
                        end

                        # --- interpolate λ to GP if needed
                        λgp = λ_is_scalarfield ? dot(λa, H[:, k]) : λconst

                        for a in 1:numNodes, b in 1:numNodes
                            ∇Na = invJ * ∇h[3a-2:3a-(3-dim), k]
                            ∇Nb = invJ * ∇h[3b-2:3b-(3-dim), k]

                            for i in 1:dim, j in 1:dim
                                ia = (a - 1) * pdim + i
                                ib = (b - 1) * pdim + j
                                Ke[ia, ib] += λgp * (Fgp[i, i] * ∇Na[i]) * (Fgp[j, j] * ∇Nb[j]) * w
                            end
                        end
                    end

                    # scatter
                    for a in 1:pdim*numNodes, b in 1:pdim*numNodes
                        I[pos] = (nodeTags[div(a - 1, pdim)+1] - 1) * pdim + mod(a - 1, pdim) + 1
                        J[pos] = (nodeTags[div(b - 1, pdim)+1] - 1) * pdim + mod(b - 1, pdim) + 1
                        V[pos] = Ke[a, b]
                        pos += 1
                    end
                end
            end
        end
    end

    resize!(I, pos - 1)
    resize!(J, pos - 1)
    resize!(V, pos - 1)

    K = sparse(I, J, V, dof, dof)
    dropzeros!(K)
    return SystemMatrix(K, problem)
end


gradDivMatrixF (generic function with 1 method)

In [1403]:
function poissonMatrixSymGradF(
    problem::Problem;
    coefficient::Union{Number,ScalarField},
    F::TensorField)

    @assert problem.pdim == problem.dim
    gmsh.model.setCurrent(problem.name)

    dim = problem.dim
    pdim = problem.pdim
    non = problem.non
    dof = pdim * non

    # --- elementwise nodal tensors for F
    Fe = nodesToElements(F)
    Fe_map = Dict(zip(Fe.numElem, Fe.A))

    # --- elementwise nodal scalars for μ (if needed)
    μ_is_scalarfield = !(coefficient isa Number)
    if μ_is_scalarfield
        μe = nodesToElements(coefficient)
        μmap = Dict(zip(μe.numElem, μe.A))  # each entry: (numNodes×1) vector per element
    else
        μconst = Float64(coefficient)
    end

    lengthOfIJV = LowLevelFEM.estimateLengthOfIJV(problem)
    I = Vector{Int}(undef, lengthOfIJV)
    J = Vector{Int}(undef, lengthOfIJV)
    V = Vector{Float64}(undef, lengthOfIJV)
    pos = 1

    for mat in problem.material
        for (edim, etag) in gmsh.model.getEntitiesForPhysicalName(mat.phName)

            elemTypes, elemTags, elemNodeTags =
                gmsh.model.mesh.getElements(edim, etag)

            for it in eachindex(elemTypes)
                et = elemTypes[it]
                _, _, order, numNodes, _, _ =
                    gmsh.model.mesh.getElementProperties(et)

                intPoints, intWeights =
                    gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))

                # --- Lagrange shape functions (for proper interpolation of F and μ)
                _, hfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, intPoints, "Lagrange")
                H = reshape(hfun, numNodes, :)

                # --- GradLagrange
                _, dfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, intPoints, "GradLagrange")
                ∇h = reshape(dfun, :, length(intWeights))

                for (e, elem) in enumerate(elemTags[it])
                    nodeTags = elemNodeTags[it][(e-1)*numNodes+1:e*numNodes]

                    jac, jacDet, _ =
                        gmsh.model.mesh.getJacobian(elem, intPoints)
                    Jac = reshape(jac, 3, :)

                    Ke = zeros(pdim * numNodes, pdim * numNodes)

                    # nodal data for this element
                    Felem = Fe_map[elem]
                    if μ_is_scalarfield
                        μa = μmap[elem][:, 1]   # length = numNodes
                    end

                    for k in eachindex(intWeights)
                        invJ = inv(Jac[1:dim, 3k-2:3k])'
                        w = jacDet[k] * intWeights[k]

                        # --- interpolate F to Gauss point using shape functions
                        Fgp = zeros(dim, dim)
                        for a in 1:numNodes
                            Na = H[a, k]
                            Fgp .+= Na * reshape(Felem[9a-8:9a, 1], dim, dim)
                        end

                        # --- interpolate μ to Gauss point if needed
                        μgp = μ_is_scalarfield ? dot(μa, H[:, k]) : μconst

                        for a in 1:numNodes, b in 1:numNodes
                            ∇Na = invJ * ∇h[3a-2:3a-(3-dim), k]
                            ∇Nb = invJ * ∇h[3b-2:3b-(3-dim), k]

                            GradNa = zeros(dim, dim)
                            GradNb = zeros(dim, dim)
                            for i in 1:dim
                                GradNa[i, i] = ∇Na[i]
                                GradNb[i, i] = ∇Nb[i]
                            end

                            Ea = 0.5 * (Fgp' * GradNa + GradNa' * Fgp)
                            Eb = 0.5 * (Fgp' * GradNb + GradNb' * Fgp)

                            for i in 1:dim, j in 1:dim
                                ia = (a - 1) * pdim + i
                                ib = (b - 1) * pdim + j
                                Ke[ia, ib] += 2 * μgp * Ea[i, j] * Eb[i, j] * w
                            end
                        end
                    end

                    # scatter
                    for a in 1:pdim*numNodes, b in 1:pdim*numNodes
                        I[pos] = (nodeTags[div(a - 1, pdim)+1] - 1) * pdim + mod(a - 1, pdim) + 1
                        J[pos] = (nodeTags[div(b - 1, pdim)+1] - 1) * pdim + mod(b - 1, pdim) + 1
                        V[pos] = Ke[a, b]
                        pos += 1
                    end
                end
            end
        end
    end

    resize!(I, pos - 1)
    resize!(J, pos - 1)
    resize!(V, pos - 1)

    K = sparse(I, J, V, dof, dof)
    dropzeros!(K)
    return SystemMatrix(K, problem)
end


poissonMatrixSymGradF (generic function with 1 method)

In [1404]:
function materialTangentTL_FC(
    problem::Problem;
    F::TensorField,
    C::AbstractMatrix
)
    @assert problem.dim == 3
    @assert problem.pdim == 3
    @assert size(C) == (6, 6)
    @assert all(x -> (x isa Number) || (x isa ScalarField), C)

    gmsh.model.setCurrent(problem.name)

    dim = 3
    pdim = 3
    dof = pdim * problem.non

    # --- elementwise nodal F (9×numNodes per element)
    Fe = nodesToElements(F)
    Fmap = Dict(zip(Fe.numElem, Fe.A))

    # --- Preprocess C entries:
    # For ScalarField entries: build element->nodal-values map once.
    Centry = Matrix{Any}(undef, 6, 6)
    for I in CartesianIndices(C)
        cij = C[I]
        if cij isa ScalarField
            ce = nodesToElements(cij)
            Centry[I] = Dict(zip(ce.numElem, ce.A))  # elem => (numNodes×1) nodal values
        else
            Centry[I] = Float64(cij)
        end
    end

    lengthOfIJV = LowLevelFEM.estimateLengthOfIJV(problem)
    I = Int[]
    J = Int[]
    V = Float64[]
    sizehint!(I, lengthOfIJV)
    sizehint!(J, lengthOfIJV)
    sizehint!(V, lengthOfIJV)

    for mat in problem.material
        for (edim, etag) in gmsh.model.getEntitiesForPhysicalName(mat.phName)

            elemTypes, elemTags, elemNodeTags =
                gmsh.model.mesh.getElements(edim, etag)

            for it in eachindex(elemTypes)
                et = elemTypes[it]
                _, _, order, numNodes, _, _ =
                    gmsh.model.mesh.getElementProperties(et)

                ip, wip =
                    gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))

                # shape functions for interpolation
                _, hfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, ip, "Lagrange")
                H = reshape(hfun, numNodes, :)

                # gradients
                _, dfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, ip, "GradLagrange")
                ∇h = reshape(dfun, :, length(wip))

                for (e, elem) in enumerate(elemTags[it])
                    nodeTags = elemNodeTags[it][(e-1)*numNodes+1:e*numNodes]

                    jac, jacDet, _ =
                        gmsh.model.mesh.getJacobian(elem, ip)
                    Jac = reshape(jac, 3, :)

                    Fnode = Fmap[elem]
                    Ke = zeros(pdim * numNodes, pdim * numNodes)

                    for k in eachindex(wip)
                        invJ = inv(Jac[1:3, 3k-2:3k])'
                        w = jacDet[k] * wip[k]

                        # --- F at Gauss: Fgp = Σ_a N_a F_a
                        Fgp = zeros(3, 3)
                        for a in 1:numNodes
                            Na = H[a, k]
                            Fgp .+= Na * reshape(Fnode[9a-8:9a, 1], 3, 3)
                        end

                        # --- C at Gauss: Cgp_ij = Σ_a N_a Cij_a  (ha ScalarField)
                        Cgp = zeros(6, 6)
                        for ii in 1:6, jj in 1:6
                            cij = Centry[ii, jj]
                            if cij isa Float64
                                Cgp[ii, jj] = cij
                            else
                                # dict elem => nodalValues(numNodes×1)
                                nod = cij[elem][:, 1]
                                Cgp[ii, jj] = dot(nod, H[:, k])
                            end
                        end

                        # --- build B(F)  (6 × 3numNodes)
                        # Column corresponds to dof (a,j): δu_{a,j}
                        # δF = e_j ⊗ ∇N_a^T
                        # δE = sym(F^T δF) = 1/2 (F^T δF + δF^T F)
                        B = zeros(6, 3 * numNodes)

                        for a in 1:numNodes
                            ∇Na = invJ * ∇h[3a-2:3a, k]  # 3-vector

                            for j in 1:3
                                # v = F^T e_j = row j of F, as a column vector
                                v = @view Fgp[j, :]  # length-3 row view

                                # dE = 1/2 (v ⊗ ∇Na^T + ∇Na ⊗ v^T)
                                dE11 = 0.5 * (v[1] * ∇Na[1] + ∇Na[1] * v[1])
                                dE22 = 0.5 * (v[2] * ∇Na[2] + ∇Na[2] * v[2])
                                dE33 = 0.5 * (v[3] * ∇Na[3] + ∇Na[3] * v[3])

                                dE23 = 0.5 * (v[2] * ∇Na[3] + ∇Na[2] * v[3])
                                dE13 = 0.5 * (v[1] * ∇Na[3] + ∇Na[1] * v[3])
                                dE12 = 0.5 * (v[1] * ∇Na[2] + ∇Na[1] * v[2])

                                col = (a - 1) * 3 + j
                                B[1, col] = dE11
                                B[2, col] = dE22
                                B[3, col] = dE33
                                B[4, col] = 2 * dE23
                                B[5, col] = 2 * dE13
                                B[6, col] = 2 * dE12
                            end
                        end

                        Ke .+= (B' * Cgp * B) * w
                    end

                    # scatter Ke
                    for a in 1:(3*numNodes), b in 1:(3*numNodes)
                        Ig = (nodeTags[div(a - 1, 3)+1] - 1) * 3 + mod(a - 1, 3) + 1
                        Jg = (nodeTags[div(b - 1, 3)+1] - 1) * 3 + mod(b - 1, 3) + 1
                        push!(I, Ig)
                        push!(J, Jg)
                        push!(V, Ke[a, b])
                    end
                end
            end
        end
    end

    K = sparse(I, J, V, dof, dof)
    dropzeros!(K)
    return SystemMatrix(K, problem)
end


materialTangentTL_FC (generic function with 1 method)

In [1405]:
function initialStressMatrixTL(
    problem::Problem;
    stress::TensorField   # P, S, σ – a kód nem tudja, nem is érdekli
)
    @assert problem.pdim == problem.dim
    gmsh.model.setCurrent(problem.name)

    dim = problem.dim
    pdim = problem.pdim
    non = problem.non
    dof = pdim * non

    # elementwise nodal stresses
    Se = nodesToElements(stress)
    Smap = Dict(zip(Se.numElem, Se.A))

    lengthOfIJV = LowLevelFEM.estimateLengthOfIJV(problem)
    I = Vector{Int}(undef, lengthOfIJV)
    J = Vector{Int}(undef, lengthOfIJV)
    V = Vector{Float64}(undef, lengthOfIJV)
    pos = 1

    for mat in problem.material
        for (edim, etag) in gmsh.model.getEntitiesForPhysicalName(mat.phName)

            elemTypes, elemTags, elemNodeTags =
                gmsh.model.mesh.getElements(edim, etag)

            for it in eachindex(elemTypes)
                et = elemTypes[it]
                _, _, order, numNodes, _, _ =
                    gmsh.model.mesh.getElementProperties(et)

                ip, wip =
                    gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))

                # --- Lagrange shape functions (for stress interpolation)
                _, hfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, ip, "Lagrange")
                H = reshape(hfun, numNodes, :)

                # --- GradLagrange
                _, dfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, ip, "GradLagrange")
                ∇h = reshape(dfun, :, length(wip))

                for (e, elem) in enumerate(elemTags[it])
                    nodeTags = elemNodeTags[it][(e-1)*numNodes+1:e*numNodes]

                    jac, jacDet, _ =
                        gmsh.model.mesh.getJacobian(elem, ip)
                    Jac = reshape(jac, 3, :)

                    Selem = Smap[elem]
                    Ke = zeros(pdim * numNodes, pdim * numNodes)

                    for k in eachindex(wip)
                        invJ = inv(Jac[1:dim, 3k-2:3k])'
                        w = jacDet[k] * wip[k]

                        # --- interpolate stress to Gauss point (CORRECT)
                        Sgp = zeros(dim, dim)
                        for a in 1:numNodes
                            Na = H[a, k]
                            Sgp .+= Na * reshape(Selem[9a-8:9a, 1], dim, dim)
                        end

                        for a in 1:numNodes, b in 1:numNodes
                            ∇Na = invJ * ∇h[3a-2:3a-(3-dim), k]
                            ∇Nb = invJ * ∇h[3b-2:3b-(3-dim), k]

                            # scalar geometric stiffness contribution
                            g = dot(∇Na, Sgp * ∇Nb)

                            # distribute to displacement components (i = i only)
                            for i in 1:dim
                                ia = (a - 1) * pdim + i
                                ib = (b - 1) * pdim + i
                                Ke[ia, ib] += g * w
                            end
                        end
                    end

                    # scatter
                    for a in 1:pdim*numNodes, b in 1:pdim*numNodes
                        I[pos] = (nodeTags[div(a - 1, pdim)+1] - 1) * pdim + mod(a - 1, pdim) + 1
                        J[pos] = (nodeTags[div(b - 1, pdim)+1] - 1) * pdim + mod(b - 1, pdim) + 1
                        V[pos] = Ke[a, b]
                        pos += 1
                    end
                end
            end
        end
    end

    resize!(I, pos - 1)
    resize!(J, pos - 1)
    resize!(V, pos - 1)

    K = sparse(I, J, V, dof, dof)
    dropzeros!(K)
    return SystemMatrix(K, problem)
end


initialStressMatrixTL (generic function with 1 method)

In [1406]:
function internalForceTL(problem::Problem, P::TensorField)

    gmsh.model.setCurrent(problem.name)

    dim = problem.dim
    pdim = problem.pdim
    non = problem.non
    dof = pdim * non

    Pe = nodesToElements(P)
    Pmap = Dict(zip(Pe.numElem, Pe.A))

    f = zeros(dof)

    for mat in problem.material
        for (edim, etag) in gmsh.model.getEntitiesForPhysicalName(mat.phName)

            elemTypes, elemTags, elemNodeTags =
                gmsh.model.mesh.getElements(edim, etag)

            for it in eachindex(elemTypes)
                et = elemTypes[it]
                _, _, order, numNodes, _, _ =
                    gmsh.model.mesh.getElementProperties(et)

                ip, wip =
                    gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))

                # --- Lagrange shape functions (for proper interpolation of P)
                _, hfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, ip, "Lagrange")
                H = reshape(hfun, numNodes, :)

                # --- GradLagrange
                _, dfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, ip, "GradLagrange")
                ∇h = reshape(dfun, :, length(wip))

                for (e, elem) in enumerate(elemTags[it])
                    nodeTags = elemNodeTags[it][(e-1)*numNodes+1:e*numNodes]

                    jac, jacDet, _ =
                        gmsh.model.mesh.getJacobian(elem, ip)
                    Jac = reshape(jac, 3, :)

                    Pelem = Pmap[elem]
                    fe = zeros(pdim * numNodes)

                    for k in eachindex(wip)
                        invJ = inv(Jac[1:dim, 3k-2:3k])'
                        w = jacDet[k] * wip[k]

                        # --- interpolate P to Gauss point using shape functions
                        Pgp = zeros(dim, dim)
                        for a in 1:numNodes
                            Na = H[a, k]
                            Pgp .+= Na * reshape(Pelem[9a-8:9a, 1], dim, dim)
                        end

                        for a in 1:numNodes
                            ∇Na = invJ * ∇h[3a-2:3a-(3-dim), k]
                            for i in 1:dim
                                ia = (a - 1) * pdim + i
                                fe[ia] += dot(Pgp[i, :], ∇Na) * w
                            end
                        end
                    end

                    # scatter
                    for a in 1:numNodes, i in 1:dim
                        Ig = (nodeTags[a] - 1) * pdim + i
                        f[Ig] += fe[(a-1)*pdim+i]
                    end
                end
            end
        end
    end

    if pdim ≠ 3
        error("dim ≠ 3 is not implemented yet")
    end
    return VectorField([], reshape(f, :, 1), [0.0], [], 1, :v3D, problem)
end


internalForceTL (generic function with 1 method)

In [1407]:
function externalTangentFollowerTL(
    problem::Problem;
    F::TensorField,
    traction_phName::AbstractString,
    t_spatial)

    gmsh.model.setCurrent(problem.name)

    dim = problem.dim
    pdim = problem.pdim
    non = problem.non
    dof = pdim * non

    Fe = nodesToElements(F)
    Fmap = Dict(zip(Fe.numElem, Fe.A))

    Id = zeros(dim, dim)
    for i in 1:dim
        Id[i, i] = 1
    end

    I = Int[]
    J = Int[]
    V = Float64[]

    for (edim, etag) in gmsh.model.getEntitiesForPhysicalName(traction_phName)
        elemTypes, elemTags, elemNodeTags =
            gmsh.model.mesh.getElements(edim, etag)

        for it in eachindex(elemTypes)
            et = elemTypes[it]
            _, _, order, numNodes, _, _ =
                gmsh.model.mesh.getElementProperties(et)

            ip, wip =
                gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))

            # Lagrange shape functions
            _, hfun, _ =
                gmsh.model.mesh.getBasisFunctions(et, ip, "Lagrange")
            H = reshape(hfun, numNodes, :)

            # GradLagrange
            _, dfun, _ =
                gmsh.model.mesh.getBasisFunctions(et, ip, "GradLagrange")
            ∇h = reshape(dfun, :, length(wip))

            for (e, elem) in enumerate(elemTags[it])
                nodeTags = elemNodeTags[it][(e-1)*numNodes+1:e*numNodes]

                jac, jacDet, _ =
                    gmsh.model.mesh.getJacobian(elem, ip)
                Jac = reshape(jac, 3, :)

                Fnode = Fmap[elem]
                Ke = zeros(pdim * numNodes, pdim * numNodes)

                for k in eachindex(wip)
                    invJ = inv(Jac[1:dim, 3k-2:3k])'
                    w = jacDet[k] * wip[k]

                    # --- interpolate F to Gauss point using shape functions
                    Fgp = zeros(dim, dim)
                    for a in 1:numNodes
                        Na = H[a, k]
                        Fgp .+= Na * reshape(Fnode[9a-8:9a, 1], dim, dim)
                    end

                    Jgp = det(Fgp)
                    Finv = inv(Fgp)
                    FinvT = Finv'

                    if t_spatial isa Function
                        tgp = t_spatial(0.0, 0.0, 0.0)
                    else
                        tgp = collect(Float64, t_spatial)
                    end

                    for a in 1:numNodes, b in 1:numNodes
                        ∇Nb = invJ * ∇h[3b-2:3b-(3-dim), k]

                        # δF from δu_bj
                        for j in 1:dim
                            A = reshape(Finv[:, j], dim, 1) *
                                reshape(∇Nb, 1, dim)

                            dJFmT =
                                Jgp * FinvT * (tr(A) * Id - A')

                            dt0 = dJFmT * tgp

                            for i in 1:dim
                                ia = (a - 1) * pdim + i
                                jb = (b - 1) * pdim + j
                                Ke[ia, jb] += H[a, k] * dt0[i] * w
                            end
                        end
                    end
                end

                # scatter Ke
                for a in 1:pdim*numNodes, b in 1:pdim*numNodes
                    Ig = (nodeTags[div(a - 1, pdim)+1] - 1) * pdim + mod(a - 1, pdim) + 1
                    Jg = (nodeTags[div(b - 1, pdim)+1] - 1) * pdim + mod(b - 1, pdim) + 1
                    push!(I, Ig)
                    push!(J, Jg)
                    push!(V, Ke[a, b])
                end
            end
        end
    end

    K = sparse(I, J, V, dof, dof)
    dropzeros!(K)
    return SystemMatrix(K, problem)
end


externalTangentFollowerTL (generic function with 1 method)

In [1408]:
@inline function _loadvec_helper(f, h, x, y, z, nnet, j, l, nsteps)
    if f isa Number
        return fill(f, nsteps)
    elseif f isa Function
        return fill(f(x, y, z), nsteps)
    elseif f isa ScalarField && isNodal(f)
        v = h[:, j]' * f.a[nnet[l, :], :]
        return vec(v)
    else
        error("loadVector: internal error.")
    end
end


_loadvec_helper (generic function with 1 method)

In [1409]:
function loadVectorNew(problem, loads; F=nothing)
    gmsh.model.setCurrent(problem.name)
    if !isa(loads, Vector)
        error("loadVector: loads are not arranged in a vector. Put them in [...]")
    end

    Fe = nothing
    Fmap = nothing
    if F !== nothing
        Fe = nodesToElements(F)
        Fmap = Dict(zip(Fe.numElem, Fe.A))
    end

    pdim = problem.pdim
    DIM = problem.dim
    b = problem.thickness
    non = problem.non
    dof = pdim * non
    ncoord2 = zeros(3 * problem.non)
    f = nothing
    fp = zeros(dof,1)
    nsteps = 1
    for n in 1:length(loads)
        name = loads[n].phName
        fx = loads[n].fx
        fy = loads[n].fy
        fz = loads[n].fz
        T = loads[n].T
        p = loads[n].p
        hc = loads[n].h
        T0 = loads[n].T
        qn = loads[n].qn
        hs = loads[n].h

        q = loads[n].q

        fxy = loads[n].fxy
        fyz = loads[n].fyz
        fzx = loads[n].fzx
        
        fyx = loads[n].fyx
        fzy = loads[n].fzy
        fxz = loads[n].fxz

        fx  = (fx  isa ScalarField && isElementwise(fx))  ? elementsToNodes(fx)  : fx
        fy  = (fy  isa ScalarField && isElementwise(fy))  ? elementsToNodes(fy)  : fy
        fz  = (fz  isa ScalarField && isElementwise(fz))  ? elementsToNodes(fz)  : fz
        T   = (T   isa ScalarField && isElementwise(T))   ? elementsToNodes(T)   : T
        p   = (p   isa ScalarField && isElementwise(p))   ? elementsToNodes(p)   : p
        hc  = (hc  isa ScalarField && isElementwise(hc))  ? elementsToNodes(hc)  : hc
        T0  = (T0  isa ScalarField && isElementwise(T0))  ? elementsToNodes(T0)  : T0
        qn  = (qn  isa ScalarField && isElementwise(qn))  ? elementsToNodes(qn)  : qn
        hs  = (hs  isa ScalarField && isElementwise(hs))  ? elementsToNodes(hs)  : hs
        q   = (q   isa ScalarField && isElementwise(q))   ? elementsToNodes(q)   : q
        fxy = (fxy isa ScalarField && isElementwise(fxy)) ? elementsToNodes(fxy) : fxy
        fyz = (fyz isa ScalarField && isElementwise(fyz)) ? elementsToNodes(fyz) : fyz
        fzx = (fzx isa ScalarField && isElementwise(fzx)) ? elementsToNodes(fzx) : fzx
        fyx = (fyx isa ScalarField && isElementwise(fyx)) ? elementsToNodes(fyx) : fyx
        fzy = (fzy isa ScalarField && isElementwise(fzy)) ? elementsToNodes(fzy) : fzy
        fxz = (fxz isa ScalarField && isElementwise(fxz)) ? elementsToNodes(fxz) : fxz
    
        nsteps = 1
        for i in [fx, fy , fz, T,  p, hc, T0, qn, hs, q, fxy, fyz, fzx, fyx, fzy, fxz]
            if i isa ScalarField
                nsteps = max(nsteps, i.nsteps)
            end
        end
        fp = zeros(dof, nsteps)

        (qn !== nothing || hc !== nothing || hs !== nothing || T !== nothing) && (fx !== nothing || fy !== nothing || fz !== nothing) &&
            error("loadVector: qn/h/T∞ and fx/fy/fz cannot be defined in the same BC.")
        if pdim == 1 || pdim == 2 || pdim == 3 || pdim == 9
            f = zeros(pdim, nsteps)
        else
            error("loadVector: dimension of the problem is $(problem.dim).")
        end
        if p !== nothing && DIM == 3 && pdim == 3
            nv = -normalVector(problem, name)
            ex = VectorField(problem, [field(name, fx=1, fy=0, fz=0)])
            ey = VectorField(problem, [field(name, fx=0, fy=1, fz=0)])
            ez = VectorField(problem, [field(name, fx=0, fy=0, fz=1)])
            if p isa Number || p isa ScalarField
                fy = elementsToNodes((nv ⋅ ey) * p)
                fz = elementsToNodes((nv ⋅ ez) * p)
                fx = elementsToNodes((nv ⋅ ex) * p)
            elseif p isa Function
                pp = scalarField(problem, [field(name, f=p)])
                fy = elementsToNodes((nv ⋅ ey) * pp)
                fz = elementsToNodes((nv ⋅ ez) * pp)
                fx = elementsToNodes((nv ⋅ ex) * pp)
            end
        end
        if p !== nothing && DIM ≠ 3
            error("loadVector: pressure can be given on a surface of a 3D solid.")
        end
        fx = fx !== nothing ? fx : 0.0
        fy = fy !== nothing ? fy : 0.0
        fz = fz !== nothing ? fz : 0.0
        dimTags = gmsh.model.getEntitiesForPhysicalName(name)
        for i ∈ 1:length(dimTags)
            dimTag = dimTags[i]
            dim = dimTag[1]
            tag = dimTag[2]
            elementTypes, elementTags, elemNodeTags = gmsh.model.mesh.getElements(dim, tag)
            nodeTags::Vector{Int64}, ncoord, parametricCoord = gmsh.model.mesh.getNodes(dim, tag, true, false)
            ncoord2[nodeTags*3 .- 2] = ncoord[1:3:length(ncoord)]
            ncoord2[nodeTags*3 .- 1] = ncoord[2:3:length(ncoord)]
            ncoord2[nodeTags*3 .- 0] = ncoord[3:3:length(ncoord)]
            for ii in 1:length(elementTypes)
                elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(elementTypes[ii])
                nnoe = reshape(elemNodeTags[ii], numNodes, :)'
                intPoints, intWeights = gmsh.model.mesh.getIntegrationPoints(elementTypes[ii], "Gauss" * string(2order+1))
                numIntPoints = length(intWeights)
                comp, fun, ori = gmsh.model.mesh.getBasisFunctions(elementTypes[ii], intPoints, "Lagrange")
                h = reshape(fun, :, numIntPoints)
                nnet = zeros(Int, length(elementTags[ii]), numNodes)
                H = zeros(pdim * numIntPoints, pdim * numNodes)
                for j in 1:numIntPoints
                    for k in 1:numNodes
                        for l in 1:pdim
                            H[j*pdim-(pdim-l), k*pdim-(pdim-l)] = h[k, j]
                        end
                    end
                end
                f1 = zeros(pdim * numNodes, nsteps)
                nn2 = zeros(Int, pdim * numNodes)
                @inbounds for l in 1:length(elementTags[ii])
                    elem = elementTags[ii][l]
                    for k in 1:numNodes
                        nnet[l, k] = elemNodeTags[ii][(l-1)*numNodes+k]
                    end
                    jac, jacDet, coord = gmsh.model.mesh.getJacobian(elem, intPoints)
                    Jac = reshape(jac, 3, :)
                    fill!(f1, 0.0)
                    @inbounds for j in 1:numIntPoints
                        x = h[:, j]' * ncoord2[nnet[l, :] * 3 .- 2]
                        y = h[:, j]' * ncoord2[nnet[l, :] * 3 .- 1]
                        z = h[:, j]' * ncoord2[nnet[l, :] * 3 .- 0]
                        if hc !== nothing && T0 !== nothing
                            if hc isa Function
                                error("heatConvectionVector: h cannot be a function.")
                            end
                            f[1,:] .= _loadvec_helper(T0, h, x, y, z, nnet, j, l, nsteps) * hc
                        elseif qn !== nothing
                            f[1,:] .= _loadvec_helper(qn, h, x, y, z, nnet, j, l, nsteps)
                        elseif hs !== nothing
                            f[1,:] .= _loadvec_helper(hs, h, x, y, z, nnet, j, l, nsteps)
                        elseif q !== nothing
                            f[1,:] .= _loadvec_helper(q, h, x, y, z, nnet, j, l, nsteps)
                        elseif fx !== nothing && pdim <= 3
                            f[1,:] .= _loadvec_helper(fx, h, x, y, z, nnet, j, l, nsteps)
                        end
                        if pdim > 1 && pdim <= 3
                            f[2,:] .= _loadvec_helper(fy, h, x, y, z, nnet, j, l, nsteps)
                        end
                        if pdim == 3
                            f[3,:] .= _loadvec_helper(fz, h, x, y, z, nnet, j, l, nsteps)
                        end
                        if pdim == 9
                            fill!(f, 0.0)
                            if fx !== nothing
                                f[1,:] .= _loadvec_helper(fx, h, x, y, z, nnet, j, l, nsteps)
                            end
                            if fy !== nothing
                                f[5,:] .= _loadvec_helper(fy, h, x, y, z, nnet, j, l, nsteps)
                            end
                            if fz !== nothing
                                f[9,:] .= _loadvec_helper(fz, h, x, y, z, nnet, j, l, nsteps)
                            end
                            if fxy !== nothing
                                f[4,:] .= _loadvec_helper(fxy, h, x, y, z, nnet, j, l, nsteps)
                            end
                            if fyz !== nothing
                                f[8,:] .= _loadvec_helper(fyz, h, x, y, z, nnet, j, l, nsteps)
                            end
                            if fzx !== nothing
                                f[3,:] .= _loadvec_helper(fzx, h, x, y, z, nnet, j, l, nsteps)
                            end
                            if fyx !== nothing
                                f[2,:] .= _loadvec_helper(fyx, h, x, y, z, nnet, j, l, nsteps)
                            end
                            if fzy !== nothing
                                f[6,:] .= _loadvec_helper(fzy, h, x, y, z, nnet, j, l, nsteps)
                            end
                            if fxz !== nothing
                                f[7,:] .= _loadvec_helper(fxz, h, x, y, z, nnet, j, l, nsteps)
                            end
                        end

                        # -------- FOLLOWER LOAD (Total Lagrange, Piola) ----------
                        if F !== nothing
                            # F elemértékek
                            Fnode = Fmap[elem]   # 9*numNodes × 1

                            # F interpoláció Gauss-pontra (pont úgy, ahogy Kext-ben)
                            Fgp = zeros(DIM, DIM)
                            for a in 1:numNodes
                                Na = h[a, j]
                                Fgp .+= Na * reshape(Fnode[9a-8:9a], DIM, DIM)
                            end

                            Jgp = det(Fgp)
                            FinvT = inv(Fgp)'

                            # Piola transzformáció a traction-re
                            for s in 1:nsteps
                                #f[:,s] .= Jgp * FinvT * f[:,s]
                            end
                        end
                        # ---------------------------------------------------------

                        r = x
                        H1 = H[j*pdim-(pdim-1):j*pdim, 1:pdim*numNodes] # H1[...] .= H[...] ????
                        ############### NANSON ######## 3D ###################################
                        if DIM == 3 && dim == 3
                            Ja = jacDet[j]
                        elseif DIM == 3 && dim == 2
                            xy = Jac[1, 3*j-2] * Jac[2, 3*j-1] - Jac[2, 3*j-2] * Jac[1, 3*j-1]
                            yz = Jac[2, 3*j-2] * Jac[3, 3*j-1] - Jac[3, 3*j-2] * Jac[2, 3*j-1]
                            zx = Jac[3, 3*j-2] * Jac[1, 3*j-1] - Jac[1, 3*j-2] * Jac[3, 3*j-1]
                            Ja = √(xy^2 + yz^2 + zx^2)
                        elseif DIM == 3 && dim == 1
                            Ja = √((Jac[1, 3*j-2])^2 + (Jac[2, 3*j-2])^2 + (Jac[3, 3*j-2])^2)
                        elseif DIM == 3 && dim == 0
                            Ja = 1
                            ############ 2D #######################################################
                        elseif DIM == 2 && dim == 2 && problem.type != :AxiSymmetric && problem.type != :AxiSymmetricHeatConduction
                            Ja = jacDet[j] * b
                        elseif DIM == 2 && dim == 2 && (problem.type == :AxiSymmetric || problem.type == :AxiSymmetricHeatConduction)
                            Ja = 2π * jacDet[j] * r
                        elseif DIM == 2 && dim == 1 && problem.type != :AxiSymmetric && problem.type != :AxiSymmetricHeatConduction
                            Ja = √((Jac[1, 3*j-2])^2 + (Jac[2, 3*j-2])^2) * b
                        elseif DIM == 2 && dim == 1 && (problem.type == :AxiSymmetric || problem.type == :AxiSymmetricHeatConduction)
                            Ja = 2π * √((Jac[1, 3*j-2])^2 + (Jac[2, 3*j-2])^2) * r
                        elseif DIM == 2 && dim == 0
                            Ja = 1
                            ############ 1D #######################################################
                        elseif DIM == 1 && dim == 1
                            Ja = Jac[1, 3*j-2] * b
                        elseif DIM == 1 && dim == 0
                            Ja = 1
                        else
                            error("loadVector: dimension of the problem is $(problem.dim), dimension of load is $dim.")
                        end
                        f1 += H1' * f * Ja * intWeights[j]
                    end
                    for k in 1:pdim
                        nn2[k:pdim:pdim*numNodes] = pdim * nnoe[l, 1:numNodes] .- (pdim - k)
                    end
                    fp[nn2,:] .+= f1
                end
            end
        end
    end
    type = :null
    if pdim == 3
        type = :v3D
    elseif pdim == 2
        type = :v2D
    elseif pdim == 1
        type = :scalar
    elseif pdim == 9
        type = :tensor
    else
        error("loadVector: wrong pdim ($pdim).")
    end
    ts = [i for i in 0:nsteps-1]
    if type == :v3D || type == :v2D
        return VectorField([], reshape(fp, :, nsteps), ts, [], nsteps, type, problem)

    elseif type == :scalar
        return ScalarField([], reshape(fp, :, nsteps), ts, [], nsteps, type, problem)

    elseif type == :tensor
        return TensorField([], reshape(fp, :, nsteps), ts, [], nsteps, type, problem)
    end
end

loadVectorNew (generic function with 1 method)

In [1410]:
function elementInternalForceTL(problem::Problem, et::Int, elem::Int, nodeTags::Vector{Int};
    P_of_F)

    dim = problem.dim
    pdim = problem.pdim
    @assert dim == 3 && pdim == 3

    _, _, order, numNodes, _, _ = gmsh.model.mesh.getElementProperties(et)
    ip, wip = gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))

    # shape & grad
    _, hfun, _ = gmsh.model.mesh.getBasisFunctions(et, ip, "Lagrange")
    H = reshape(hfun, numNodes, :)
    _, dfun, _ = gmsh.model.mesh.getBasisFunctions(et, ip, "GradLagrange")
    ∇h = reshape(dfun, :, length(wip))

    # coords of element nodes (reference)
    # gmsh returns node coords globally; itt feltételezem van segédfüggvényed:
    # X = getNodeCoords(problem, nodeTags) :: (3×numNodes)
    X = nodeCoordinates(problem, nodeTags)  # <-- ha nálad más a neve, írd át

    fe = zeros(pdim * numNodes)

    jac, jacDet, _ = gmsh.model.mesh.getJacobian(elem, ip)
    Jac = reshape(jac, 3, :)

    for k in eachindex(wip)
        invJ = inv(Jac[1:dim, 3k-2:3k])'
        w = jacDet[k] * wip[k]

        # GradN in reference
        GradN = zeros(3, numNodes)
        for a in 1:numNodes
            GradN[:, a] = invJ * ∇h[3a-2:3a, k]
        end

        # deformation gradient from current u (u is encoded in current nodal positions x = X + u)
        # Here we assume caller already provides current nodal positions x (or u_e).
        # We'll compute F = x ⊗ GradN + I  form:
        # F = sum_a x_a ⊗ GradN_a   (with x_a current position vector)
        # Caller will provide current x in a closure; see next function.

        # We'll leave F computation to closure by passing (X, GradN, k) if you want.
        # For simplicity: P_of_F is expected to be called in the Ke-builder where x is known.

        # This function will be used from Ke-builder where we know x.
        # So here we just return fe placeholder.
    end

    return fe
end


elementInternalForceTL (generic function with 1 method)

In [1411]:
function elementMaterialTangentFD_TL(problem::Problem, et::Int, elem::Int, nodeTags::Vector{Int};
    P_of_F,
    h=1e-8)

    dim = problem.dim
    pdim = problem.pdim
    @assert dim == 3 && pdim == 3

    _, _, order, numNodes, _, _ = gmsh.model.mesh.getElementProperties(et)
    ip, wip = gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))

    _, hfun, _ = gmsh.model.mesh.getBasisFunctions(et, ip, "Lagrange")
    H = reshape(hfun, numNodes, :)
    _, dfun, _ = gmsh.model.mesh.getBasisFunctions(et, ip, "GradLagrange")
    ∇h = reshape(dfun, :, length(wip))

    X = nodeCoordinates(problem, nodeTags)  # 3×numNodes reference coords

    jac, jacDet, _ = gmsh.model.mesh.getJacobian(elem, ip)
    Jac = reshape(jac, 3, :)

    # fe(u_e) black-box
    function fe_of_ue(ue::Vector{Float64})
        # current positions x = X + u
        x = copy(X)
        for a in 1:numNodes
            x[:, a] .+= ue[(a-1)*3+1:(a-1)*3+3]
        end

        fe = zeros(3 * numNodes)

        for k in eachindex(wip)
            invJ = inv(Jac[1:dim, 3k-2:3k])'
            w = jacDet[k] * wip[k]

            # GradN in reference
            GradN = zeros(3, numNodes)
            for a in 1:numNodes
                GradN[:, a] = invJ * ∇h[3a-2:3a, k]
            end

            # F = sum_a x_a ⊗ GradN_a
            F = zeros(3, 3)
            for a in 1:numNodes
                F .+= x[:, a] * (GradN[:, a])'
            end

            P = P_of_F(F)  # 3×3

            # fe_a += (P^T * GradN_a) * w   (equiv to P : ∇N)
            # Your internalForceTL used: fe[ia] += dot(P[i,:], ∇Na) * w
            for a in 1:numNodes
                t = P * GradN[:, a]  # vector length 3
                ia = (a - 1) * 3
                fe[ia+1] += t[1] * w
                fe[ia+2] += t[2] * w
                fe[ia+3] += t[3] * w
            end
        end

        return fe
    end

    ndofe = 3 * numNodes
    Ke = zeros(ndofe, ndofe)

    ue0 = zeros(ndofe)  # caller will replace with actual element displacement when assembling
    f0 = fe_of_ue(ue0)

    for j in 1:ndofe
        ue = copy(ue0)
        ue[j] += h
        fj = fe_of_ue(ue)
        Ke[:, j] = (fj - f0) / h
    end

    return Ke, f0
end


elementMaterialTangentFD_TL (generic function with 1 method)

In [1412]:
function materialTangentFD_TL(problem::Problem, u::VectorField; P_of_F, h=1e-8)

    gmsh.model.setCurrent(problem.name)

    dim = problem.dim
    pdim = problem.pdim
    dof = pdim * problem.non

    I = Int[]
    J = Int[]
    V = Float64[]
    f = zeros(dof)

    for mat in problem.material
        for (edim, etag) in gmsh.model.getEntitiesForPhysicalName(mat.phName)

            elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(edim, etag)

            for it in eachindex(elemTypes)
                et = elemTypes[it]
                _, _, _, numNodes, _, _ = gmsh.model.mesh.getElementProperties(et)

                for (e, elem) in enumerate(elemTags[it])
                    nodeTags = elemNodeTags[it][(e-1)*numNodes+1:e*numNodes]

                    # extract element displacement ue from global u
                    ue = zeros(3 * numNodes)
                    for a in 1:numNodes
                        g = nodeTags[a]
                        ue[(a-1)*3+1:(a-1)*3+3] .= u.a[(g-1)*3+1:(g-1)*3+3, 1]
                    end

                    # build Ke and fe at this ue
                    Ke, fe0 = elementMaterialTangentFD_TL(problem, Int(et), Int(elem), Vector{Int}(nodeTags); P_of_F=P_of_F, h=h)

                    # scatter fe
                    for a in 1:numNodes
                        g = nodeTags[a]
                        f[(g-1)*3+1:(g-1)*3+3] .+= fe0[(a-1)*3+1:(a-1)*3+3]
                    end

                    # scatter Ke
                    for a in 1:3*numNodes, b in 1:3*numNodes
                        Ig = (nodeTags[div(a - 1, 3)+1] - 1) * 3 + mod(a - 1, 3) + 1
                        Jg = (nodeTags[div(b - 1, 3)+1] - 1) * 3 + mod(b - 1, 3) + 1
                        push!(I, Ig)
                        push!(J, Jg)
                        push!(V, Ke[a, b])
                    end
                end
            end
        end
    end

    K = sparse(I, J, V, dof, dof)
    dropzeros!(K)

    return SystemMatrix(K, problem), VectorField([], reshape(f, :, 1), [0.0], [], 1, :v3D, problem)
end


materialTangentFD_TL (generic function with 1 method)

In [1413]:
gmsh.initialize()
gmsh.open("cube1.geo")

mat = Material("cube", E=2e2)
prob = Problem([mat], type=:VectorField)

Info    : Reading 'cube1.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 60%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 70%] Meshing curve 9 (Line)
Info    : [ 80%] Meshing curve 10 (Line)
Info    : [ 90%] Meshing curve 11 (Line)
Info    : [100%] Meshing curve 12 (Line)
Info    : Done meshing 1D (Wall 0.00131855s, CPU 0.000951s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Transfinite)
Info    : [ 20%] Meshing surface 2 (Transfinite)
Info    : [ 40%] Meshing surface 3 (Transfinite)
Info    : [ 60%] Meshing surface 4 (Transfinite)
Info    : [ 70%] Meshing surface 5 (Transfinite)
Info    : [ 90%] Meshing surface 6 (Transfinite)
Info    : Done meshing 2D (Wall 0.00179319s, CPU 0.000693s)
Info    : Meshing 3D..

Problem("cube1", :VectorField, 3, 3, Material[Material("cube", :Hooke, 200.0, 0.3, 115.38461538461537, 76.92307692307692, 166.66666666666663, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 1.0e-7, 0.1, 1.0)], 1.0, 6859, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))

In [1414]:
F = TensorField(prob, "cube", [1 0 0; 0 1 0; 0 0 1]);

In [1415]:
#Fleft = nodesToElements(elementsToNodes(F), onPhysicalGroup="left")

#Kext = externalTangentFollowerTL(prob; F=Fleft, traction_phName="left", t_spatial=[1, 0, 0])

In [1416]:
function check_fint_Kint_consistency(
    problem,
    u::VectorField;
    bc,
    C,
    ε=1e-7,
    verbose=true
)

    # --- aktuális állapot ---
    F = u ∘ ∇
    I = unitTensor(F)
    E = (F' * F - I) / 2
    S = λ * trace(E) * I + 2μ * E
    P = F * S

    f0 = internalForceTL(problem, P).a[:, 1]

    # --- tangens ---
    #Kmat =
    #    poissonMatrixSymGradF(problem; coefficient=μ, F=F).A +
    #    gradDivMatrixF(problem; coefficient=λ, F=F).A

    #Kgeo = initialStressMatrixTL(problem; stress=P).A
    Kint = materialTangentTL_FC(problem, F=F, C=C)

    # --- perturbáció csak szabad DoF-okon ---
    fixed = constrainedDoFs(problem, bc)
    free = setdiff(1:length(f0), fixed)

    δu = zeros(length(f0))
    δu[free] .= randn(length(free))
    δu ./= norm(δu)

    # --- numerikus derivált ---
    u_pert = copy(u)
    u_pert.a[:, 1] .+= ε .* δu

    #Fp = deformationGradient(problem, u_pert)
    Fp = u_pert ∘ ∇
    Ep = (Fp' * Fp - I) / 2
    Sp = λ * trace(Ep) * I + 2μ * Ep
    Pp = Fp * Sp

    f1 = internalForceTL(problem, Pp).a[:, 1]

    df_fd = (f1 - f0) / ε
    df_lin = Kint.A * δu

    rel_err = norm(df_fd - df_lin) / max(norm(df_fd), 1e-14)

    if verbose
        println("‖df_fd‖  = ", norm(df_fd))
        println("‖Kδu‖    = ", norm(df_lin))
        println("rel err = ", rel_err)
    end

    return rel_err
end


check_fint_Kint_consistency (generic function with 1 method)

In [1634]:
suppX = BoundaryCondition("left", ux=0, uy=0, uz=0)
suppY = BoundaryCondition("bottom", uy=0)
suppZ = BoundaryCondition("rear", uz=0)

load1 = BoundaryCondition("frontright", fy=-4)
load2 = BoundaryCondition("rearright", fy=4)

BoundaryCondition("rearright", nothing, nothing, nothing, nothing, nothing, nothing, nothing, 4, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing)

In [1418]:
μ = mat.μ
λ = mat.λ

115.38461538461537

In [1419]:
f_ext0 = loadVector(prob, [load1]) + loadVector(prob, [load2])

nodal VectorField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [1420]:
showDoFResults(f_ext0, name="f")

114

In [1421]:
u = vectorField(prob, "cube", [0, 0, 0])

nodal VectorField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [1422]:
F = tensorField(prob, "cube", [1 0 0; 0 1 0; 0 0 1])

nodal TensorField
[1.0; 0.0; … ; 0.0; 1.0;;]

In [1423]:
I = tensorField(prob, "cube", [1 0 0; 0 1 0; 0 0 1])

nodal TensorField
[1.0; 0.0; … ; 0.0; 1.0;;]

In [1424]:
P = tensorField(prob, "cube", [0 0 0; 0 0 0; 0 0 0])
S = tensorField(prob, "cube", [0 0 0; 0 0 0; 0 0 0])

nodal TensorField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [1425]:
λ0 = λ
μ0 = μ
C1 = [λ0+2μ0 λ0 λ0 0 0 0;
    λ0 λ0+2μ0 λ0 0 0 0;
    λ0 λ0 λ0+2μ0 0 0 0;
    0 0 0 μ0 0 0;
    0 0 0 0 μ0 0;
    0 0 0 0 0 μ0]

6×6 Matrix{Float64}:
 269.231  115.385  115.385   0.0      0.0      0.0
 115.385  269.231  115.385   0.0      0.0      0.0
 115.385  115.385  269.231   0.0      0.0      0.0
   0.0      0.0      0.0    76.9231   0.0      0.0
   0.0      0.0      0.0     0.0     76.9231   0.0
   0.0      0.0      0.0     0.0      0.0     76.9231

## Innen indul az iteráció

In [1648]:
#Kμ = poissonMatrixSymGradF(prob, coefficient=μ, F=F)
#Kλ = gradDivMatrixF(prob, coefficient=λ, F=F)

#Kmat = Kμ + Kλ
Kmat = materialTangentTL_FC(prob, F=F, C=C1)

Kgeo = initialStressMatrixTL(prob, stress=S)

Kint = Kmat + Kgeo

sparse([1, 2, 3, 46, 47, 48, 73, 74, 75, 76  …  20511, 20521, 20522, 20523, 20569, 20570, 20571, 20575, 20576, 20577], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577], [7.170827228558487, 0.5588639284675848, -0.8609625618519373, -0.40730010087560137, -0.19085503553033406, -0.03397265443098059, -2.1833591532916956, 0.4365323077134298, 0.16319360732834515, -0.4095780556929438  …  -12.491571504919586, -3.535311915953856, -100.38037784107713, -109.43759532403173, -0.016786074483039287, -0.07067222977113653, 73.57665047428941, 5.253318796943471, -0.911502946196536, 583.655671854223], 20577, 20577)

In [1649]:
Ffrontright = nodesToElements(elementsToNodes(F), onPhysicalGroup="frontright")
Frearright = nodesToElements(elementsToNodes(F), onPhysicalGroup="rearright")

Kext1 = externalTangentFollowerTL(
    prob;
    F=Ffrontright,
    traction_phName="frontright",
    t_spatial=[0.0, -4.0, 0.0]
)

Kext2 = externalTangentFollowerTL(
    prob;
    F=Frearright,
    traction_phName="rearright",
    t_spatial=[0.0, 4.0, 0.0]
)

sparse(Int64[], Int64[], Float64[], 20577, 20577)

In [1650]:
f_int = internalForceTL(prob, P)

nodal VectorField
[0.00036881032638697324; 0.0003506798297926687; … ; -2.890199691200782e-5; 1.2043722143794192e-5;;]

In [1651]:
K = Kint - Kext1 - Kext2

sparse([1, 2, 3, 46, 47, 48, 73, 74, 75, 76  …  20511, 20521, 20522, 20523, 20569, 20570, 20571, 20575, 20576, 20577], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577], [7.170827228558487, 0.5588639284675848, -0.8609625618519373, -0.40730010087560137, -0.19085503553033406, -0.03397265443098059, -2.1833591532916956, 0.4365323077134298, 0.16319360732834515, -0.4095780556929438  …  -12.491571504919586, -3.535311915953856, -100.38037784107713, -109.43759532403173, -0.016786074483039287, -0.07067222977113653, 73.57665047428941, 5.253318796943471, -0.911502946196536, 583.655671854223], 20577, 20577)

In [1652]:
FF = nodesToElements(elementsToNodes(F), onPhysicalGroup="frontright")
FR = nodesToElements(elementsToNodes(F), onPhysicalGroup="rearright")
f_ext = loadVectorNew(prob, [loadF], F=FF) + loadVectorNew(prob, [loadR], F=FR)


nodal VectorField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [1653]:
#showDoFResults(f_ext, name="f_ext")
#showDoFResults(f_int, name="f_int")

In [1654]:
Δu = solveField(
    K,
    f_ext - f_int;
    support=[suppX]
)

fixed = constrainedDoFs(prob, [suppX])
free = freeDoFs(prob, [suppX])
#norm(Δu.a[:, 1])
r = f_ext - f_int
println("‖r_free‖  = ", norm(r.a[free, 1]))
println("‖r_fixed‖ = ", norm(r.a[fixed, 1]))
println("‖r‖ = ", norm(r.a[:, 1]))
println("‖Δu‖ = ", norm(Δu.a[:, 1]))


‖r_free‖  = 0.0003930200181028268
‖r_fixed‖ = 0.18296945827932304
‖r‖ = 0.18296988038407733
‖Δu‖ = 0.0008302875470958339


In [1655]:
u += Δu

nodal VectorField
[0.0; 0.0; … ; -0.4468357837925049; 0.18458997139438066;;]

In [1656]:
showDoFResults(u)
openPostProcessor()

XRequest.18: BadValue 0x0


-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


In [1657]:
F = I + u ∘ ∇

elementwise TensorField
[[0.9975247105122186; 0.028209368780472595; … ; -0.04207345083305217; 0.9985441162161658;;], [0.9972827644738811; 0.03479514285924651; … ; -0.13616122150263388; 0.9906503429276785;;], [0.9972327217069427; 0.03791793408659222; … ; -0.22764726118316791; 0.9737809001917852;;], [0.9972234772785028; 0.040199919519192384; … ; -0.3170019784034615; 0.9484731991505861;;], [0.9972227105474689; 0.042104280252413195; … ; -0.40361159088352505; 0.9149835728894918;;], [0.9972154825788798; 0.04365374753134865; … ; -0.4867386267931546; 0.8736175761741742;;], [0.9971414378730648; 0.04486150186565496; … ; -0.5656249752373528; 0.8248382590383843;;], [0.9966735589733746; 0.04582722419883223; … ; -0.6391232407122248; 0.7698975609936661;;], [0.999723930244215; 0.05550507478642763; … ; -0.7020529360102001; 0.7114346497640414;;], [0.9949821207492259; 0.028959300746167912; … ; -0.04281600048666809; 0.99925464291132;;]  …  [1.0021659179443012; -0.05276270146916673; … ; -0.7041117271496501

In [1658]:
E = 0.5 * (F' * F - I)
S = λ * trace(E) * I + 2μ * E
P = F * S
#P = λ * trace(F - I) * I + 2μ * (F - I)

elementwise TensorField
[[-0.4422249352103301; 2.1574456027915767; … ; -0.02727826203192043; -0.0763845589020607;;], [-0.39935951356693655; 1.3910213918487593; … ; -0.036078873647510265; -0.004016334738035049;;], [-0.39400968153249133; 1.5279564015271332; … ; -0.03821258822028803; 0.0029798055076348615;;], [-0.39557719198981006; 1.6438722189032475; … ; -0.03875356225817422; 0.0006671595632067745;;], [-0.395435859106131; 1.726960391847243; … ; -0.03876090393910249; -0.002338232336134751;;], [-0.3952249547651565; 1.7923285436628877; … ; -0.03884617602946131; -0.002634449699928863;;], [-0.4082070895812616; 1.8491347473403938; … ; -0.04222063537803652; 0.01571728621525736;;], [-0.5414378826389796; 1.9250379262386133; … ; -0.06590774608529319; 0.14500747513313766;;], [0.0663958820665397; 2.7197860948634194; … ; -0.09160258786022203; 0.8018060391136621;;], [-1.1688023682379751; 2.193620119420282; … ; -0.028802012389153978; -0.01303087604517437;;]  …  [0.3228823989264466; -1.3030233337914612;

In [1646]:
err = check_fint_Kint_consistency(prob, u; C=C1, bc=[suppX])

println("Relative tangent error = ", err)


‖df_fd‖  = 391.89123974772764
‖Kδu‖    = 90.15391101108442
rel err = 1.0846841173758306
Relative tangent error = 1.0846841173758306


In [1647]:
#gmsh.finalize()